# Tutorial 1: Part 2

Objectives:
- Learn how to define a simple lattice and compute the TWISS functions using MAD-X.
- Thick vs thin lense approximation TWISS comparison for a lattice with only quadrupoles.
- Tune and $\beta$-function dependence on K1.

**My first accelerator: a FODO cell**

1. Make a simple lattice cell with $L_{cell}$= 100 m made of focusing and defocusing qudrupoles (so called FODO cell). Each quad is 5 m long ($L_{quad}$). Put the start of the first qudrupole at the start of the sequence. Each quadrupole has a focal length f=200 m (k1 x $L_{quad}$ = 1/f in thin lense approximation).


2. Define a proton beam at $E_{tot}$= 2 GeV. Activate the sequence and try to find the periodic solution and plot the $\beta$-functions. If you found $\beta_{max}$= 460 m you succeded.


3. Using the plot you obtained can you estimate the phase advance of the cell. Compare with the tunes obtained with the TWISS.


4. Try with $E_{tot}$ = 0.7 GeV: what is the MAD-X error message?


5. Try with f= 20 m: what is the MAD-x error message?

**Matching the FODO cell using a parametric plot**

6. Try to twiss it powering the quds to obtain a $\Delta \mu \approx 90^\circ$ in the cell using the thin lense approximation (using the figures from Tutorial 1: Part 1). What is the actual phase advance computed by MAD-X?


7. What is the $\beta_{mx}$? Compare with the thin lense approximation (using the figures from Tutorial 1: Part 1).


8. Compute the maximum beam size $\sigma$ assuming a normalized emittance of 3 mrad mm and $E_{tot}= 7 TeV$.


9. Halve the focusing strength of the quadrupoles, wht is the effect of it on the $\beta_{max}$ and $\beta_{min}$ and on the $\Delta \mu$? Compare with the parametric plots.

In [4]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from cpymad.madx import Madx 

# Launching MAD-X

In [5]:
myMad = Madx()


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.07.00  (64 bit, Darwin)      +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2021.05.03               +
  + Execution date: 2021.11.26 17:37:03      +
  ++++++++++++++++++++++++++++++++++++++++++++


# Definition of parameters

In [9]:
myMad = Madx()
myString='''
! *********************************************************************
! Definition of parameters
! *********************************************************************

l_cell=100;
quadrupoleLenght=5;
f=200;
myK:=1/f/quadrupoleLenght;// m^-2

! *********************************************************************
! Definition of magnets
! ********************************************************************* 
QF: quadrupole, L=quadrupoleLenght, K1:=myK;
QD: quadrupole, L=quadrupoleLenght, K1:=-myK;

! *********************************************************************
! Definition of sequence
! *********************************************************************
myCell:sequence, refer=start, L=L_CELL;
quadrupole1: QF, at=0;
marker1: marker, at=25;
quadrupole2: QD, at=50;
endsequence;

! *********************************************************************
! Definition of beam
! *********************************************************************
beam, particle=proton, energy=2;

! *********************************************************************
! Use of the sequence
! *********************************************************************
use, sequence=myCell;

! *********************************************************************
! TWISS
! *********************************************************************
title, 'My first twiss';
twiss;

'''
myMad.input(myString);


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.07.00  (64 bit, Darwin)      +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2021.05.03               +
  + Execution date: 2021.11.26 17:38:50      +
  ++++++++++++++++++++++++++++++++++++++++++++


RuntimeError: MAD-X has stopped working!

# Accessing the SUMM and TWISS table results

In [ ]:
myString='''
value, table(SUMM,Q1);
value, table(SUMM,betymax);
'''
myMad.input(myString);

In [ ]:
myString='''
value, table(TWISS,QUADRUPOLE2,S);
value, table(TWISS,QUADRUPOLE2,BETX);
'''
myMad.input(myString);

In [ ]:
myString='''
value, table(TWISS,mycell$end:1,Re11);
value, table(TWISS,mycell$end:1,Re12);
value, table(TWISS,mycell$end:1,Re21);
value, table(TWISS,mycell$end:1,Re22);
'''
myMad.input(myString);

In [ ]:
112.125/np.sin(0.25)

# Accessing table with pandas
For advance users...

Pandas dataframe are very convenient, have a look in https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf.

In [ ]:
myDF=myMad.table.twiss.dframe()
myDF[['name','s','betx','bety']]

In [ ]:
# Plot
plt.plot(myDF['s'],myDF['betx'],'ob-')
plt.plot(myDF['s'],myDF['bety'],'or-')
plt.legend()
plt.grid()
plt.xlabel('s [m]')
plt.ylabel('[m]')
plt.title('My first FODO cell')

In [ ]:
print(list(myMad.elements))

In [ ]:
print(list(myMad.sequence))

In [ ]:
myDF

In [ ]:
myDF[['name','keyword','s','betx','bety']]

# Tune from the plot

For the phase advance one can consider the definition

\begin{equation}
\mu=\int\frac{1}{\beta(s)}ds.
\end{equation}

Remember that the unit of phase in MAD-X is [2$\pi$].

In [ ]:
# a very basic approximation
1/400.*100/2/np.pi

In [ ]:
# computing the integral
np.trapz(1/myDF['betx'],myDF['s'])/2/np.pi

In [ ]:
# correct values
myDF.iloc[-1]['mux']

# Varying the energy

In [ ]:
# with this simple wrapper in case of error the code enter in an infinite loop you have to stop manually.
myString='''
beam, particle=proton, energy=0.7;
title, 'My third twiss';
twiss, file=./secondpart/thirdTwiss.txt;
'''
myMad.input(myString);

There is an error due to the fact that the total energy.


# With f=20 m

In [ ]:
myMad = Madx()

In [ ]:
myString='''
! *********************************************************************
! Tutorial 1, second part
! *********************************************************************

! *********************************************************************
! Definition of parameters
! *********************************************************************

l_cell=100;
quadrupoleLenght=5;
f=20;
myK:=1/f/quadrupoleLenght;// m^-2

! *********************************************************************
! Definition of magnet
! ********************************************************************* 
QF: quadrupole, L=quadrupoleLenght, K1:=myK;
QD: quadrupole, L=quadrupoleLenght, K1:=-myK;


! *********************************************************************
! Definition of sequence
! *********************************************************************
myCell:sequence, refer=entry, L=L_CELL;
quadrupole1: QF, at=0;
marker1: marker, at=25;
quadrupole2: QD, at=50;
endsequence;

! *********************************************************************
! Definition of beam
! *********************************************************************
beam, particle=proton, energy=2;

! *********************************************************************
! Use of the sequence
! *********************************************************************
use, sequence=myCell;

! *********************************************************************
! TWISS
! *********************************************************************
select, flag=twiss, clear;
select, flag=twiss, column=name, keyword, s, betx, alfx, mux, bety, alfy, muy, x, px, y, py, dx, dy, dx, dpx, dy, dpy;

title, 'My fourth twiss';
twiss, file=./secondpart/fourthTwiss.txt;
'''
myMad.input(myString);

INTERPRETATION: The cell is unstable due to the fact that the focal length is too short. Please note the value of the cosmux and cosmuy. 

# Adding markers

This is an example to add markers in the sequence using a macros.


In [ ]:
myString='''
! *********************************************************************
! Tutorial 1, second part
! *********************************************************************

! *********************************************************************
! Definition of parameters
! *********************************************************************

option, echo=false, info=false, warn=false;
l_cell=100;
quadrupoleLenght=5;
f=200;
myK:=1/f/quadrupoleLenght;// m^-2

! *********************************************************************
! Definition of magnet
! ********************************************************************* 
QF: quadrupole, L=quadrupoleLenght, K1:=myK;
QD: quadrupole, L=quadrupoleLenght, K1:=-myK;


installMarkers(nn): macro ={
markernn: marker, at=nn;
!value,f;
};
N=6;


! *********************************************************************
! Definition of sequence
! *********************************************************************
myCell:sequence, REFER=centre, L=L_CELL;
quadrupole1: QF, at=2.5;
while (N<50) {
exec, installMarkers($N);
N=N+1;
}
quadrupole2: QD, at=52.5;
N=56;
while (N<100) {
exec, installMarkers($N);
N=N+1;
}
endsequence;

! *********************************************************************
! Definition of beam
! *********************************************************************
beam, particle=proton, energy=2;

! *********************************************************************
! Use of the sequence
! *********************************************************************
use, sequence=myCell;

! *********************************************************************
! TWISS
! *********************************************************************
select, flag=twiss, clear;
select, flag=twiss, column=name, keyword, L, s, betx, alfx, mux, bety, alfy, muy, x, px, y, py, dx, dy, dx, dpx, dy, dpy;

title, 'My fourth twiss';
twiss, file=./secondpart/fifthTwiss.txt;
'''
myMad.input(myString);

In [ ]:
myDF=myMad.table.twiss.dframe()
myDF.head()

In [ ]:
plt.plot(myDF['s'],myDF['betx'],'.-b',label='$\\beta_x$')
plt.plot(myDF['s'],myDF['bety'],'.-r',label='$\\beta_y$')
plt.xlabel('s [m]')
plt.ylabel('[m]')
plt.legend(loc='best')
plt.grid()

In [ ]:
np.trapz(1/myDF['betx'],myDF['s'])/2/np.pi

# Thin lense approximation

In [ ]:
myString='''
select, flag=makethin, class=quadrupole, slice=5; 
makethin,sequence=myCell;
use,sequence=myCell;      
twiss,sequence=myCell,file=./secondpart/sixthTwiss.txt;
'''
myMad.input(myString);

In [ ]:
myDF=myMad.table.twiss.dframe()
myDF

In [ ]:
plt.plot(myDF['s'],myDF['betx'],'.-b',label='$\\beta_x$')
plt.plot(myDF['s'],myDF['bety'],'.-r',label='$\\beta_y$')
plt.xlabel('s [m]')
plt.ylabel('[m]')
plt.legend(loc='best')
plt.grid()

In [ ]:
np.trapz(1/myDF['betx'],myDF['s'])/2/np.pi